<a href="https://colab.research.google.com/github/jieun0441/XMC/blob/main/FMC_Code12_calculate_PE3%2C_PE7%2C_SC1%2C_SC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Setting to use a GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Setting to Use a High-RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 185.7 gigabytes of available RAM

You are using a high-RAM runtime!


일원2동이 개포3동으로 명칭이 변경되면서 동 코드도 변경된 사실을 확인.
이에 일원2동 동 코드 1168740으로 기재되어 있었던 경우, 개포3동 동코드 1168075로 변경해서 합쳐줄 필요 있음

In [ ]:
# dong code 중에 개포3동(11680675)이 몇개 있고, 일원2동(11680740)이 몇개 있는지 확인
import pandas as pd
# 1월 데이터 중 사이즈 작은 파일 하나를 활용해서 어떤 컬럼명 확인
dfM01_p5 = pd.read_csv("/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01/purpose_5_with_OD_indicators.csv")
dfM01_p5.columns

Index(['O_ADMDONG_CD', 'D_ADMDONG_CD', 'ST_TIME_CD', 'FNS_TIME_CD',
       'IN_FORN_DIV_NM', 'FORN_CITIZ_NM', 'MOVE_PURPOSE', 'MOVE_DIST',
       'MOVE_TIME', 'CNT', 'ETL_YMD', 'date', 'O_Seoul', 'D_Seoul', 'O_SMA',
       'D_SMA'],
      dtype='object')

In [ ]:
#  각 월(M01~M12) 폴더의 purpose_5_with_OD_indicators.csv 파일에서
# 'O_ADMDONG_CD'와 'D_ADMDONG_CD' 컬럼에 대해 각각 11680675(개포3동)와 11680740(일원2동)이 몇 번 등장하는지 카운트하는 코드
import os
import pandas as pd

base_path = '/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents'
months = [f'M{str(i).zfill(2)}' for i in range(1, 13)]
target_codes = [11680675, 11680740]
result = []

for month in months:
    file_path = os.path.join(base_path, month, 'purpose_5_with_OD_indicators.csv')
    if not os.path.exists(file_path):
        print(f"{file_path} 파일이 없습니다.")
        continue

    df = pd.read_csv(file_path, usecols=['O_ADMDONG_CD', 'D_ADMDONG_CD'])
    for code in target_codes:
        o_count = (df['O_ADMDONG_CD'] == code).sum()
        d_count = (df['D_ADMDONG_CD'] == code).sum()
        result.append({'Month': month, 'Code': code, 'O_ADMDONG_CD_Count': o_count, 'D_ADMDONG_CD_Count': d_count})

# 결과를 데이터프레임으로 보기 좋게 출력
result_df = pd.DataFrame(result)
print(result_df)

   Month      Code  O_ADMDONG_CD_Count  D_ADMDONG_CD_Count
0    M01  11680675                   0                   0
1    M01  11680740                  23                   0
2    M02  11680675                  21                   0
3    M02  11680740                   0                   0
4    M03  11680675                  20                   0
5    M03  11680740                   0                   0
6    M04  11680675                  22                   0
7    M04  11680740                   0                   0
8    M05  11680675                  20                   0
9    M05  11680740                   0                   0
10   M06  11680675                  15                   0
11   M06  11680740                   0                   0
12   M07  11680675                  22                   0
13   M07  11680740                   0                   0
14   M08  11680675                  19                   0
15   M08  11680740                   0                  

확인 결과, 위 결과처럼 1월에만 일원2동의 코드가 사용되었던 것으로 확인됨.
이에 일원2동 코드를 개포3동 코드로 변경 후 Index 계산 진행함.
아래 코드는 M01 폴더 안에 있는 모든 7개 파일에 대해 O_ADMDONG_CD와 D_ADMDONG_CD 값이 11680740이면 11680675로 바꿔서 덮어쓰는 코드

In [ ]:
import os
import pandas as pd

folder = '/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01'
purpose_files = [f'purpose_{i}_with_OD_indicators.csv' for i in range(1, 8)]  # 1~7

for fname in purpose_files:
    file_path = os.path.join(folder, fname)
    if not os.path.exists(file_path):
        print(f'파일 없음: {file_path}')
        continue

    df = pd.read_csv(file_path)
    # 값 변경
    df['O_ADMDONG_CD'] = df['O_ADMDONG_CD'].replace(11680740, 11680675)
    df['D_ADMDONG_CD'] = df['D_ADMDONG_CD'].replace(11680740, 11680675)
    # 덮어쓰기 저장
    df.to_csv(file_path, index=False)
    print(f'변경 및 저장 완료: {file_path}')

변경 및 저장 완료: /content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01/purpose_1_with_OD_indicators.csv
변경 및 저장 완료: /content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01/purpose_2_with_OD_indicators.csv
변경 및 저장 완료: /content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01/purpose_3_with_OD_indicators.csv
변경 및 저장 완료: /content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01/purpose_4_with_OD_indicators.csv
변경 및 저장 완료: /content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01/purpose_5_with_OD_indicators.csv
변경 및 저장 완료: /content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01/purpose_6_with_OD_indicators.csv
변경 및 저장 완료: /content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/M01/purpose_7_with_OD_indicators.csv


# Calculate PE & SC with 2023 1 year data

- base_path: 데이터가 있는 최상위 경로를 지정합니다.
- months: 'M01'부터 'M12'까지 월별 폴더명을 리스트로 만듭니다.
- use_cols: 최종 데이터프레임에 남길 컬럼만 리스트로 지정합니다.
- 각 월 폴더에서 목적별 파일을 찾아 읽고, 필요 없는 컬럼을 제거한 뒤 리스트에 저장합니다.
- 마지막에 pd.concat()으로 리스트의 데이터프레임들을 하나로 병합합니다.
- 파일이 없을 경우 메시지를 출력합니다.

In [ ]:
import pandas as pd
import os
#@title Nonwork intra Seoul trip data merge
# 경로 및 설정
base_path = '/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents'
months = [f'M{str(i).zfill(2)}' for i in range(1, 13)]
# 필요한 컬럼만 남깁니다
use_cols = [
    'O_ADMDONG_CD', 'D_ADMDONG_CD', 'ST_TIME_CD', 'FNS_TIME_CD',
    'MOVE_PURPOSE', 'MOVE_DIST', 'MOVE_TIME', 'CNT', 'ETL_YMD',
    'O_Seoul', 'D_Seoul', 'O_SMA', 'D_SMA'
]
nonwork_purposes = [2, 4, 5, 6, 7]

df_nonwork_intraSeoul_list = []

for month in months:
    month_path = os.path.join(base_path, month)
    for p in nonwork_purposes:
        file_path = os.path.join(month_path, f'purpose_{p}_with_OD_indicators.csv')
        if os.path.exists(file_path):
            # 필요한 컬럼만 읽어서 메모리 절약
            df = pd.read_csv(file_path, usecols=use_cols)
            # intra-Seoul 조건 바로 적용
            df = df[(df['O_Seoul']==1) & (df['D_Seoul']==1)]
            df_nonwork_intraSeoul_list.append(df)
        else:
            print(f"파일 없음: {file_path}")

# concat 후 바로 저장 (메모리 절약 위해 df 만들고 바로 저장)
df_nonwork_intraSeoul = pd.concat(df_nonwork_intraSeoul_list, ignore_index=True)
df_nonwork_intraSeoul.to_csv('/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/df_nonwork_intraSeoul.csv', index=False)
#del df_nonwork_intraSeoul, df_nonwork_intraSeoul_list
print('df_nonwork_intraSeoul 저장 완료')

df_nonwork_intraSeoul 저장 완료


In [ ]:
#@title Non-working trip PE SC 계산
import pandas as pd
import os

# 저장 경로 설정
save_folder = '/content/drive/MyDrive/FMC_Data/PE_and_SC_indices_Table'

# ====== Non-working trip (intra-Seoul) ======
# df_nonwork_intraSeoul = pd.read_csv('/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/df_nonwork_intraSeoul.csv')

# --- Pull Effect Index (PE3) ---
total_cnt_nonwork = df_nonwork_intraSeoul['CNT'].sum()
pe_nonwork = (
    df_nonwork_intraSeoul.groupby('D_ADMDONG_CD')['CNT'].sum()
    .reset_index().rename(columns={'CNT': 'Numerator_PE3_CNT'})
)
pe_nonwork['PE3_Seoul'] = (pe_nonwork['Numerator_PE3_CNT'] / total_cnt_nonwork) * 100

# 저장
pe_nonwork.to_csv(os.path.join(save_folder, 'PE3_nonwork_by_dong_in_Seoul.csv'), index=False)

In [ ]:
# --- Self-Containment Index (SC1) ---
# Numerator: same dong
same_dong_nonwork = df_nonwork_intraSeoul[df_nonwork_intraSeoul['O_ADMDONG_CD'] == df_nonwork_intraSeoul['D_ADMDONG_CD']]
numerator_sc1 = same_dong_nonwork.groupby('O_ADMDONG_CD')['CNT'].sum().rename('Within_Same_Dong')
# Denominator: all trips from origin dong
denominator_sc1 = df_nonwork_intraSeoul.groupby('O_ADMDONG_CD')['CNT'].sum().rename('Total_Origin_Trips')
# Merge and calculate
sc1_nonwork = pd.concat([numerator_sc1, denominator_sc1], axis=1).fillna(0)
sc1_nonwork['SC1'] = 100 * sc1_nonwork['Within_Same_Dong'] / sc1_nonwork['Total_Origin_Trips']
sc1_nonwork = sc1_nonwork.reset_index().rename(columns={'O_ADMDONG_CD': 'ADMDONG_CD'})
# 저장
sc1_nonwork.to_csv(os.path.join(save_folder, 'SC1_nonwork_by_dong_in_Seoul.csv'), index=False)

In [ ]:
pe_nonwork

,D_ADMDONG_CD,Numerator_PE3_CNT,PE3_Seoul
0,11110515,6429809.42,0.259180
1,11110530,12896501.83,0.519847
2,11110540,3242548.04,0.130704
3,11110550,3311255.99,0.133474
4,11110560,3609338.72,0.145489
...,...,...,...
421,11740650,5052847.21,0.203676
422,11740660,7684242.39,0.309745
423,11740685,10452905.55,0.421348
424,11740690,1278338.60,0.051529


In [ ]:
total_cnt_nonwork

np.float64(2480825904.869981)

In [ ]:
#@title Work intra Seoul trip data merge
import pandas as pd
import os

base_path = '/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents'
months = [f'M{str(i).zfill(2)}' for i in range(1, 13)]
use_cols = [
    'O_ADMDONG_CD', 'D_ADMDONG_CD', 'ST_TIME_CD', 'FNS_TIME_CD',
    'MOVE_PURPOSE', 'MOVE_DIST', 'MOVE_TIME', 'CNT', 'ETL_YMD',
    'O_Seoul', 'D_Seoul', 'O_SMA', 'D_SMA'
]
work_purposes = [1]

df_work_intraSeoul_list = []

for month in months:
    month_path = os.path.join(base_path, month)
    for p in work_purposes:
        file_path = os.path.join(month_path, f'purpose_{p}_with_OD_indicators.csv')
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, usecols=use_cols)
            df = df[(df['O_Seoul']==1) & (df['D_Seoul']==1)]
            df_work_intraSeoul_list.append(df)
        else:
            print(f"파일 없음: {file_path}")

df_work_intraSeoul = pd.concat(df_work_intraSeoul_list, ignore_index=True)
df_work_intraSeoul.to_csv('/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents/df_work_intraSeoul.csv', index=False)
#del df_work_intraSeoul, df_work_intraSeoul_list
print('df_work_intraSeoul 저장 완료')

df_work_intraSeoul 저장 완료


In [ ]:
import pandas as pd
import os

# 저장 경로 설정
save_folder = '/content/drive/MyDrive/FMC_Data/PE_and_SC_indices_Table'

#@title Working trip PE SC 계산
# ====== Working trip (intra-Seoul) ======
#df_work_intraSeoul = pd.read_csv(os.path.join(save_folder, 'df_work_intraSeoul.csv'))

# --- Pull Effect Index (PE7) ---
total_cnt_work = df_work_intraSeoul['CNT'].sum()
pe_work = (
    df_work_intraSeoul.groupby('D_ADMDONG_CD')['CNT'].sum()
    .reset_index().rename(columns={'CNT': 'Numerator_CNT'})
)
pe_work['PE7_Seoul'] = (pe_work['Numerator_CNT'] / total_cnt_work) * 100

# 저장
pe_work.to_csv(os.path.join(save_folder, 'PE7_work_by_dong_in_Seoul.csv'), index=False)

# --- Self-Containment Index (SC2) ---
same_dong_work = df_work_intraSeoul[df_work_intraSeoul['O_ADMDONG_CD'] == df_work_intraSeoul['D_ADMDONG_CD']]
numerator_sc2 = same_dong_work.groupby('O_ADMDONG_CD')['CNT'].sum().rename('Within_Same_Dong')
denominator_sc2 = df_work_intraSeoul.groupby('O_ADMDONG_CD')['CNT'].sum().rename('Total_Origin_Trips')
sc2_work = pd.concat([numerator_sc2, denominator_sc2], axis=1).fillna(0)
sc2_work['SC2'] = 100 * sc2_work['Within_Same_Dong'] / sc2_work['Total_Origin_Trips']
sc2_work = sc2_work.reset_index().rename(columns={'O_ADMDONG_CD': 'ADMDONG_CD'})
sc2_work.to_csv(os.path.join(save_folder, 'SC2_work_by_dong_in_Seoul.csv'), index=False)

In [ ]:
# df_PE3 = pd.read_csv('/content/drive/MyDrive/FMC_Data/PE3_nonwork_by_dong_in_Seoul.csv')
# df_PE3.columns

Index(['D_ADMDONG_CD', 'Numerator_PE3_CNT', 'PE3_Seoul'], dtype='object')

In [ ]:
# df_PE7 = pd.read_csv('/content/drive/MyDrive/FMC_Data/PE7_work_by_dong_in_Seoul.csv')
# df_PE7.columns

Index(['D_ADMDONG_CD', 'Numerator_CNT', 'PE7_Seoul'], dtype='object')

In [ ]:
# df_SC1 = pd.read_csv('/content/drive/MyDrive/FMC_Data/SC1_nonwork_by_dong_in_Seoul.csv')
# df_SC1.columns

Index(['ADMDONG_CD', 'Within_Same_Dong', 'Total_Origin_Trips', 'SC1'], dtype='object')

In [ ]:
# df_SC2 = pd.read_csv('/content/drive/MyDrive/FMC_Data/SC2_work_by_dong_in_Seoul.csv')
# df_SC2.columns

Index(['ADMDONG_CD', 'Within_Same_Dong', 'Total_Origin_Trips', 'SC2'], dtype='object')

In [ ]:
# 저장 경로 설정
save_folder = '/content/drive/MyDrive/FMC_Data/PE_and_SC_indices_Table'

# Each entry: (filename, code_column, index_column)
index_files = [
    ('PE3_nonwork_by_dong_in_Seoul.csv', 'D_ADMDONG_CD', 'PE3_Seoul'),
    ('PE7_work_by_dong_in_Seoul.csv', 'D_ADMDONG_CD', 'PE7_Seoul'),
    ('SC1_nonwork_by_dong_in_Seoul.csv', 'ADMDONG_CD', 'SC1'),
    ('SC2_work_by_dong_in_Seoul.csv', 'ADMDONG_CD', 'SC2')
]

# Read and prepare each DataFrame
dfs = []
for fname, code_col, index_col in index_files:
    path = os.path.join(save_folder, fname)
    df = pd.read_csv(path, usecols=[code_col, index_col])
    df = df.rename(columns={code_col: 'ADMDONG_CD', index_col: index_col})  # standardize key name
    dfs.append(df)

# Merge all on 'ADMDONG_CD'
from functools import reduce
combined_df = reduce(lambda left, right: pd.merge(left, right, on='ADMDONG_CD', how='outer'), dfs)
combined_df.to_csv(os.path.join(save_folder, 'Seoul_ADMDONG_indices_combined.csv'), index=False)
combined_df

,ADMDONG_CD,PE3_Seoul,PE7_Seoul,SC1,SC2
0,11110515,0.259180,0.154843,19.764209,12.682711
1,11110530,0.519847,0.867687,15.432103,26.444967
2,11110540,0.130704,0.091754,8.050622,12.606470
3,11110550,0.133474,0.070045,19.036250,9.270658
4,11110560,0.145489,0.081985,19.939680,9.782003
...,...,...,...,...,...
421,11740650,0.203676,0.165426,17.916287,10.460896
422,11740660,0.309745,0.239511,20.348462,13.106206
423,11740685,0.421348,0.358944,26.846764,15.981620
424,11740690,0.051529,0.037705,9.235021,4.615486


# 참고: 기존 PE1~8 계산식1

In [ ]:
#@title PE1: nonworking(24567) PE(Seoul) (i != j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = [
    'purpose_2_with_OD_indicators.csv',
    'purpose_4_with_OD_indicators.csv',
    'purpose_5_with_OD_indicators.csv',
    'purpose_6_with_OD_indicators.csv',
    'purpose_7_with_OD_indicators.csv'
]

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for O_Seoul==1, D_Seoul==1, and O_ADMDONG_CD != D_ADMDONG_CD
df_seoul = df[
    (df['O_Seoul'] == 1) &
    (df['D_Seoul'] == 1) &
    (df['O_ADMDONG_CD'] != df['D_ADMDONG_CD'])
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE1_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE1_within_Seoul_to_Seoul_24567_btw_notsameOD.csv'), index=False)

In [ ]:
#@title PE2: nonworking(24567) PE(Korea) (i != j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = [
    'purpose_2_with_OD_indicators.csv',
    'purpose_4_with_OD_indicators.csv',
    'purpose_5_with_OD_indicators.csv',
    'purpose_6_with_OD_indicators.csv',
    'purpose_7_with_OD_indicators.csv'
]

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for D_Seoul==1, and O_ADMDONG_CD != D_ADMDONG_CD
df_all_to_seoul = df[
    (df['D_Seoul'] == 1) &
    (df['O_ADMDONG_CD'] != df['D_ADMDONG_CD'])
]

# Total count for denominator
total_cnt_PE2 = df_all_to_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator_PE2 = df_all_to_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe2_df = numerator_PE2.reset_index().rename(columns={'CNT': 'Numerator_PE2_CNT'})
pe2_df['PE_all_to_seoul'] = (pe2_df['Numerator_PE2_CNT'] / total_cnt_PE2) * 100

# rename the PE column (Because we have different versions of PE)
pe2_df = pe2_df.rename(columns={'PE_all_to_seoul': 'PE2_all_to_seoul'})
# Save result (optional)
pe2_df.to_csv(os.path.join(folder, 'PE2_all_to_seoul_24567_btw_notsameOD.csv'), index=False)

In [ ]:
#@title PE3: nonworking(24567) PE(Seoul) (including i=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M_B_dong_data_by_purpose_domestic_residents'
files = [
    'purpose_2_with_OD_indicators.csv',
    'purpose_4_with_OD_indicators.csv',
    'purpose_5_with_OD_indicators.csv',
    'purpose_6_with_OD_indicators.csv',
    'purpose_7_with_OD_indicators.csv'
]

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for O_Seoul==1, D_Seoul==1
df_seoul = df[
    (df['O_Seoul'] == 1) & (df['D_Seoul'] == 1)
]

# Total count for denominator
total_cnt_PE3 = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator_PE3 = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe3_df = numerator_PE3.reset_index().rename(columns={'CNT': 'Numerator_PE3_CNT'})
pe3_df['PE_Seoul'] = (pe3_df['Numerator_PE3_CNT'] / total_cnt_PE3) * 100

# rename the PE column (Because we have different versions of PE)
pe3_df = pe3_df.rename(columns={'PE_Seoul': 'PE3_Seoul'})
# Save result (optional)
pe3_df.to_csv(os.path.join(folder, 'PE3_within_Seoul_to_Seoul_24567_including_all_OD.csv'), index=False)

In [ ]:
# check the sum of PE3_Seoul
print(pe3_df['PE3_Seoul'].sum())

99.99999999999983


In [ ]:
#@title PE4: nonworking(24567) PE(Korea) (including i=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = [
    'purpose_2_with_OD_indicators.csv',
    'purpose_4_with_OD_indicators.csv',
    'purpose_5_with_OD_indicators.csv',
    'purpose_6_with_OD_indicators.csv',
    'purpose_7_with_OD_indicators.csv'
]

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for D_Seoul==1
df_seoul = df[(df['D_Seoul'] == 1)]

# Total count for denominator
total_cnt_PE4 = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator_PE4 = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe4_df = numerator_PE4.reset_index().rename(columns={'CNT': 'Numerator_PE4_CNT'})
pe4_df['PE_Seoul'] = (pe4_df['Numerator_PE4_CNT'] / total_cnt_PE4) * 100

# rename the PE column (Because we have different versions of PE)
pe4_df = pe4_df.rename(columns={'PE_Seoul': 'PE4_Seoul'})
# Save result (optional)
pe4_df.to_csv(os.path.join(folder, 'PE4_All_to_Seoul_24567_including_all_OD.csv'), index=False)

In [ ]:
# check the sum of PE4_Seoul
print(pe4_df['PE4_Seoul'].sum())

99.99999999999994


In [ ]:
#@title PE5: working trip(1) PE(Seoul) (i!=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = ['purpose_1_with_OD_indicators.csv']

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for O_Seoul==1, D_Seoul==1, and O_ADMDONG_CD != D_ADMDONG_CD
df_seoul = df[
    (df['O_Seoul'] == 1) &
    (df['D_Seoul'] == 1) &
    (df['O_ADMDONG_CD'] != df['D_ADMDONG_CD'])
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE5_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE5_within_Seoul_to_Seoul_1working_btw_notsameOD.csv'), index=False)

In [ ]:
# check the sum of PE5_Seoul
print(pe_df['PE5_Seoul'].sum())

99.99999999999996


In [ ]:
#@title PE6: working trip(1) PE(Korea) (i!=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = ['purpose_1_with_OD_indicators.csv']

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for D_Seoul==1, and O_ADMDONG_CD != D_ADMDONG_CD
df_seoul = df[
    (df['D_Seoul'] == 1) &
    (df['O_ADMDONG_CD'] != df['D_ADMDONG_CD'])
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE6_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE6_all_to_Seoul_1working_btw_notsameOD.csv'), index=False)

In [ ]:
# check the sum of PE6_Seoul
print(pe_df['PE6_Seoul'].sum())

99.99999999999996


In [ ]:
#@title PE7: working trip(1) PE(Seoul to Seoul) (including i=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = ['purpose_1_with_OD_indicators.csv']

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for O_Seoul == 1 & D_Seoul==1
df_seoul = df[
    (df['O_Seoul'] == 1) &
    (df['D_Seoul'] == 1)
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE7_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE7_Seoul_to_Seoul_1working_btw_allOD.csv'), index=False)

In [ ]:
# check the sum of PE7_Seoul
print(pe_df['PE7_Seoul'].sum())

100.0


In [ ]:
#@title PE8: working trip(1) PE(Korea to Seoul) (including i=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = ['purpose_1_with_OD_indicators.csv']

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for D_Seoul==1
df_seoul = df[
    (df['D_Seoul'] == 1)
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE8_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE8_Seoul_to_Seoul_1working_btw_allOD.csv'), index=False)


In [ ]:
# check the sum of PE7_Seoul
print(pe_df['PE8_Seoul'].sum())

100.00000000000007
